In [2]:
%matplotlib inline
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [3]:
# READ PATHS
# Get prediction bounding boxes
prediction_path = '../../../Unicamp_PDI/resultados.txt'#'../AB1_BC1_result.txt' #results/ab1/AB1_CONTOUR_OTSU.txt' # caminho para a pasta 
prediction = []
F = open(prediction_path,'r')
for line in F.readlines():
    name = line.split()[0]
    bbox = np.array(line.split()[1:],'int')
    prediction.append(np.hstack((name,bbox)))
F.close()
# Get ground truth bounding boxes
ground_truth_path = '../all_plates_position.txt' # caminho para a pasta 
ground_truth = []
F = open(ground_truth_path,'r')
for line in F.readlines():
    name = line.split()[0]
    bbox = np.array(line.split()[1:],'int')
    ground_truth.append(np.hstack((name,bbox)))
F.close()

In [4]:
# Looking for vector
print 'Prediction'
print 'Number of plates = ', len(prediction)
print prediction[0:5]
print 'Ground Truth'
print 'Number of plates = ', len(prediction)
print ground_truth[0:5]

Prediction
Number of plates =  2000
[array(['69[04', '376', '697', '96', '26'], dtype='|S21'), array(['69[01', '424', '680', '103', '24'], dtype='|S21'), array(['69[06', '340', '709', '110', '27'], dtype='|S21'), array(['69[15', '1077', '1047', '86', '33'], dtype='|S21'), array(['69[13', '1273', '96', '101', '35'], dtype='|S21')]
Ground Truth
Number of plates =  2000
[array(['69[04', '382', '698', '113', '37'], dtype='|S21'), array(['69[02', '416', '688', '105', '35'], dtype='|S21'), array(['69[01', '430', '684', '107', '31'], dtype='|S21'), array(['69[14', '164', '776', '141', '47'], dtype='|S21'), array(['69[17', '64', '818', '165', '51'], dtype='|S21')]


In [5]:
def sortlist(char,elem):
    xs = []
    for array in char:
        xs.append(array[elem])
    xs = np.array(xs)
    ind = np.argsort(xs)
    temp = np.array(char)
    return temp[ind]

#Define auxiliary IoU fuctions
def calc_intersection_area(preditic_bbox,groundTruth_bbox):
    # convert prediction bbox to two points rect representation
    xp1 = preditic_bbox[0]
    yp1 = preditic_bbox[1]
    xp2 = preditic_bbox[0]+preditic_bbox[2]
    yp2 = preditic_bbox[1]+preditic_bbox[3]
    
    # convert Ground Truth bbox to two points rect representation
    xg1 = groundTruth_bbox[0]
    yg1 = groundTruth_bbox[1]
    xg2 = groundTruth_bbox[0]+groundTruth_bbox[2]
    yg2 = groundTruth_bbox[1]+groundTruth_bbox[3]
    
    # determine the coordinates of the intersection rectangle
    x_left = max(xp1, xg1)
    y_top = max(yp1, yg1)
    x_right = min(xp2, xg2)
    y_bottom = min(yp2, yg2)

    # verify 
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    
    return (x_right - x_left) * (y_bottom - y_top)

def calc_union_area(preditic_bbox,groundTruth_bbox,intersection_Area):
    prediction_Area = preditic_bbox[2]*preditic_bbox[3]
    groundTruth_Area = groundTruth_bbox[2]*groundTruth_bbox[3]
    return prediction_Area + groundTruth_Area - intersection_Area
    
#Define IoU fuction
def calc_IoU(preditic_bbox,groundTruth_bbox):
    Aint = calc_intersection_area(preditic_bbox,groundTruth_bbox)
    Auni = calc_union_area(preditic_bbox,groundTruth_bbox,Aint)
    return 1.0 * Aint/Auni

In [6]:
s_pred = sortlist(prediction, 0)
s_groundTruth = sortlist(ground_truth,0)

for i in range(0,2000,200):
    print s_pred[i], s_groundTruth[i]

['00[01' '539' '659' '99' '35'] ['00[01' '538' '660' '99' '33']
['17[22' '214' '832' '148' '51'] ['17[22' '226' '838' '135' '41']
['27[12' '1512' '694' '122' '40'] ['27[12' '1512' '690' '119' '43']
['38[08' '647' '992' '200' '88'] ['38[08' '434' '668' '105' '35']
['48[12' '415' '705' '116' '40'] ['48[12' '414' '702' '117' '43']
['58[19' '1656' '778' '141' '50'] ['58[19' '1663' '781' '132' '46']
['69[07' '321' '715' '144' '29'] ['69[07' '328' '720' '119' '37']
['79[04' '476' '690' '127' '39'] ['79[04' '484' '694' '103' '35']
['88[10' '63' '377' '100' '32'] ['88[10' '1538' '756' '147' '53']
['98[19' '1633' '722' '118' '43'] ['98[19' '1632' '720' '117' '43']


In [7]:
iou_scores = []
thr = 0.5
iou_rst = 0
for i in range(len(s_groundTruth)):
    score = calc_IoU(s_pred[i][1:].astype(int),s_groundTruth[i][1:].astype(int))
    #print s_pred[i][1:].astype(int)
    iou_scores.append(score)
    if(score >= thr):
        iou_rst += 1
print 'IOU result = ',iou_rst/20.0,'%'

IOU result =  73.35 %


In [8]:
poor = 0
good = 0
excellent = 0
wrong = 0
for scr in iou_scores:
    if(scr <= 0):
        wrong += 1
    elif scr < 0.7:
        poor += 1
    elif scr < 0.9:
        good += 1
    else:
        excellent += 1
print 'WRONG = ',wrong/20.0,'%'
print 'POOR = ',poor/20.0,'%'
print 'GOOD = ',good/20.0,'%'
print 'EXCELLENT = ',excellent/20.0,'%'

WRONG =  25.0 %
POOR =  8.2 %
GOOD =  43.95 %
EXCELLENT =  22.85 %
